# <center> Modelo de Markowitz para Seleção de Portfólio com Rebalanceamento Diário<center>

In [1]:
# required libraries
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.optimize import Bounds
from scipy.optimize import minimize
from tqdm import tqdm
import yfinance as yf

In [2]:
# Global parameters
ASSETS = pd.Series(['VALE3', 'ITUB4', 'PETR4', 'BBDC4', 'B3SA3', 'PETR3', 'MGLU3', 'ABEV3', 'WEGE3', 'ITSA4', 'BBAS3', 'PCAR3'])
AMOUNT_ASSETS = len(ASSETS) #number of assets to create the portfolio
LEN_TRAIN = 60 #number of past days to train the model
NUM_FRONT_POINT = 4 #amount of number to construct the Pareto front
FRONT_POINTS = np.linspace(0, 1, NUM_FRONT_POINT) #uniform distribution of the weights used in the optimization 
                                                  #to build the Pareto frontier
LEN_SLIDING_WINDOW = 248 #stop criterion
INITIAL_WEALTH = 100000 # portfolio initial wealth

START_2020 = '2020-10-01' #the model takes 60 days for training
                          #the training is repeated daily, it uses the most recent information available through
                          #the sliding window
END_2020 = '2021-12-30'

In [3]:
ibovespa = yf.download(tickers=['^BVSP'], start=START_2020, end = END_2020, rounding=True)['Adj Close']
df_ret_ibovespa_2020 = ibovespa.pct_change()[1:]

[*********************100%***********************]  1 of 1 completed


In [4]:
prices = yf.download(tickers=(ASSETS + '.SA').tolist(), start=START_2020, end= END_2020, rounding=True)['Adj Close']
prices.columns = prices.columns.str.rstrip('.SA')  # removes the .SA from the asset's name
df_ret_2020 = prices.pct_change()[1:]
#df_ret_2020.fillna(method="backfill",inplace = True)

[*********************100%***********************]  12 of 12 completed


In [6]:
ibovespa['2020-09-30']

94603.0

# TO BE DEVELOPED:
## Ensure data consistency by comparing ibovespa and  prices DFs
The 2020 version of this program works with 2 DFs, price(308,12) and ibovespa(308). We want to keep them this size.

In [14]:
#
#  df = prices.merge(ibovespa, how = 'outer', indicator=True).loc[lambda x : x['_merge']=='left_only']
prices_dates = prices.T

In [ ]:
# # Alternative -------------------------------------
# # The line code below does the trick for 2 dataframes (for more info visit https://kanoki.org/2019/07/04/pandas-difference-between-two-dataframes/)

# #  df = df1.merge(df2, how = 'outer' ,indicator=True).loc[lambda x : x['_merge']=='left_only']

# # another post suggests something different ( https://stackoverflow.com/questions/53645882/pandas-merging-101)

# (left.merge(right, on='key', how='right', indicator=True)
#      .query('_merge == "right_only"')
#      .drop('_merge', 1))


# Constraints

In [ ]:
# Definition of the investment bound constraint
bounds = Bounds(np.zeros(AMOUNT_ASSETS), np.ones(AMOUNT_ASSETS))

- Remember that, in general, we only work with the constraint `$≤$`.
- So it is necessary to make an **“adaptation”** in the way of writing the constraints defined by `$≥$`.

In [ ]:
# b1    <= A * x   <==>   -b1 >= -A*x        <==>   A*x - b1 >= 0
# A * x <= b2      <==>    A*x - b2 <= 0     <==>  -Ax + b2 >= 0
A = np.ones(AMOUNT_ASSETS)
b = np.array([1])

cons = [{"type": "ineq", "fun": lambda x: A @ x - b}, {"type": "ineq", "fun": lambda x: -A @ x + b}]

In [ ]:
# initial condition: the naive portfolio
U0 = 1/AMOUNT_ASSETS*np.ones(AMOUNT_ASSETS)

In [ ]:
# Check if any rows are missing between df_ret_2020 and df_ret_ibovespa

# Optimization method

In [ ]:
# Defines the objective functions (return and risk)
# Takes the weight to be used in the weighted combination using the parameter aux_ponto_front
# Returns the optimal portfolio came from the weight optimization considering the weight aux_dot_front
def get_optimize(aux_ponto_front, aux_train_return):
    
    def f(U,front_point = aux_ponto_front,train_return = aux_train_return):
        #train_return=returns[0:LEN_TRAIN-1]
        mean_return = train_return.mean()
        portfolio_return = mean_return.values.dot(U)
        portfolio_risk = (U.dot(np.cov(train_return.T))).dot(U)
        return front_point*portfolio_risk-(1-front_point)*portfolio_return

    Umin = minimize(f, U0, args=(),method='SLSQP',  jac=None, bounds=bounds, constraints=cons, 
                    tol=None,callback=None,options={'maxiter': 1000, 'ftol': 1e-16, 'iprint': 0,'eps': 1e-8})    
    return Umin.x

In [ ]:
# Gets the portfolio return and risk obtained by the optimal portfolio
def get_return_risk(portfolio,train_return):
    mean_return = train_return.mean()
    portfolio_return = mean_return.values.dot(portfolio)
    portfolio_risk = (portfolio.dot(np.cov(train_return.T))).dot(portfolio)
    return portfolio_return, portfolio_risk

In [ ]:
# Runs the problem
#Input data: the DataFrame daily return
#Output data: the daily optimal portfolio for the complete period

#Proceding: at each time instant k:
# Step_1: the model is trained using the last 60 past days.
# Step_2: the weighted optimization w*portfolio_risk-(1-w)*portfolio_return is runned using the SLSQP method.
    # the parameter w is a weight in [0,1]
    # a Pareto front is generated
#Step_3: chose a Pareto front point to be applied to the system
    # we chose the Pareto front midpoint, which is the investment adviser to a moderate profile investor
    # another choice criterion can be setted
#Step_4: Updates the system with the newest information from the real world
    # the oldest information is discarded
#Step_5: if the stop criterion is done
            #break
        #else
            #go to Step_1
def run_problem(df_ret):
    num_otim = 0
    selected_portfolios = pd.DataFrame([], index=np.arange(0, AMOUNT_ASSETS), columns=['Day_' + str(day) for day in range(LEN_SLIDING_WINDOW)])
    for k in tqdm(range(LEN_TRAIN, LEN_TRAIN+ LEN_SLIDING_WINDOW)):
        df_train = df_ret[k - LEN_TRAIN:k-1]
        front_portfolios = []
        for front_point in FRONT_POINTS:
            portfolio = get_optimize(front_point, df_train)
            front_portfolios.append(portfolio) 
        pos = int(NUM_FRONT_POINT/2) #choose the Pareto front midpoint ----> Moderate profile   
        selected_portfolio = front_portfolios[pos]
        selected_portfolios['Day_' + str(num_otim)] = pd.DataFrame(np.matrix(selected_portfolio)).T
        num_otim +=1
        print(k)
    selected_portfolios.index = df_ret.columns
    selected_portfolios.columns = df_ret[LEN_TRAIN-1:LEN_TRAIN+LEN_SLIDING_WINDOW-1].index    
    return selected_portfolios

In [ ]:
# Uses the daily optimal portfolio to define the portfolio daily rentability 
def get_rentability(df_ret, selected_portfolios):
    dates = selected_portfolios.columns
    cont = 0
    rentability = pd.DataFrame([], index = dates, columns = ['Rentability'])
    for k in range(LEN_TRAIN, LEN_TRAIN+LEN_SLIDING_WINDOW):
        next_day_return = df_ret[k-1:k]
        rentability['Rentability'][cont] = next_day_return.values[0].dot(selected_portfolios[dates[cont]].values)
        cont +=1
    return rentability 

In [ ]:
# Get's the cumulative wealth
def get_cumulative_wealth(rentability):
    cum_rent = pd.DataFrame([], index = rentability.index, columns = ['Cumulative Wealth'])
    cum_rent['Cumulative Wealth'] = INITIAL_WEALTH*np.cumprod(1 + rentability['Rentability'].values) - 1
    return cum_rent

In [ ]:
#Runs the problem for 2020
selected_portfolios_2020 = run_problem(df_ret_2020)
rentability_2020 = get_rentability(df_ret_2020, selected_portfolios_2020)
aux_cumulative_wealth_2020 = get_cumulative_wealth(rentability_2020)
first_day_2020 = pd.DataFrame([INITIAL_WEALTH], index =['2019-12-30 00:00:00'], columns = ['Cumulative Wealth'])
cumulative_wealth_2020 = first_day_2020.append(aux_cumulative_wealth_2020)

In [ ]:
# Get's the Ibovespa cumulative wealth for 2020
cumulative_wealth_2020.index = pd.to_datetime(cumulative_wealth_2020.index)
aux_cumulative_ibovespa_2020 = np.append(INITIAL_WEALTH,INITIAL_WEALTH*np.cumprod(1 + df_ret_ibovespa_2020[LEN_TRAIN-1:].values) - 1)
cumulative_ibovespa_2020 = pd.DataFrame([], index = df_ret_ibovespa_2020.index[LEN_TRAIN-2:], columns = ['Cumulative Wealth'])
cumulative_ibovespa_2020['Cumulative Wealth'] = aux_cumulative_ibovespa_2020

In [ ]:
plt.figure(figsize = (15,7))
plt.plot(cumulative_wealth_2020['Cumulative Wealth'],label="Markowitz",color = 'darkslategray',linestyle = (0, (5, 1)), linewidth = 4)
plt.plot(cumulative_ibovespa_2020['Cumulative Wealth'],label="Ibovespa",color = 'darkslategray',linestyle = ':', linewidth = 4)
legend = plt.legend(loc='lower right', shadow=False, fontsize= 18,  ncol = 2)
plt.xlabel('Sliding windows days', fontsize=20)
plt.ylabel('Wealth', fontsize=20)
plt.rc('xtick', labelsize=20) 
plt.rc('ytick', labelsize=20) 
plt.show()